In [1]:
import pandas as pd

In [2]:
p279DF = pd.read_csv('../data/P279_dataset/P279_Sim_Dataset_all_annotated.csv')

In [3]:
p279DF['node1_label'] = p279DF.node1_label.apply(lambda p: p.replace('\\',''))
p279DF['node2_label'] = p279DF.node2_label.apply(lambda p: p.replace('\\',''))

In [4]:
p279DF.head()

,ID,node1,node1_label,node2,node2_label,relation_type,P_Sim,P_Dim,H_Sim,H_Dim,...,S_Sim,S_Dim,N_Sim,N_Dim,F_Sim,F_Dim,J_Sim,J_Dim,Average,StdDev
0,1,Q48842375,Asian Women's Softball Championship,Q1079023,championship,parent/child,2,i,2,I,...,2,i,2,I,2,I,3,I,2.142857,0.377964
1,2,Q8001434,Wildlife of South Africa,Q104021,fauna,parent/child,2,i,2,I,...,2,i,3,H,2,I,3,I,2.285714,0.487950
2,3,Q40189366,interstellar radiation field,Q11379,energy,parent/child,3,i,3,I,...,2,i,3,H,3,D,3,I,2.714286,0.487950
3,4,Q2162453,traditional political party,Q7278,political party,parent/child,1,NaN,2,I,...,2,i,2,I,2,I,2,I,1.714286,0.487950
4,5,Q6786579,materials physics,Q413,physics,parent/child,2,i,2,I,...,2,i,3,H,2,I,2,H,2.142857,0.377964


# Split to ChildPar, Siblings Datasets

In [5]:
p279_ChildParDF = p279DF[p279DF.relation_type == 'parent/child']
p279_SiblingsDF = p279DF[p279DF.relation_type == 'sibling']

In [6]:
p279_SiblingsDF.head()

,ID,node1,node1_label,node2,node2_label,relation_type,P_Sim,P_Dim,H_Sim,H_Dim,...,S_Sim,S_Dim,N_Sim,N_Dim,F_Sim,F_Dim,J_Sim,J_Dim,Average,StdDev
25,26,Q4165229,gain,Q570938,net income,sibling,2,i,2,I,...,2,i,3,D,2,S,2,I,2.142857,0.377964
26,27,Q30461,president,Q48352,head of state,sibling,1,NaN,2,I,...,2,i,2,I,1,NaN,2,I,1.571429,0.534522
27,28,Q381072,crisis,Q3839081,disaster,sibling,2,i,2,I,...,2,i,1,NaN,1,NaN,2,I,1.571429,0.534522
28,29,Q39546,tool,Q7397,software,sibling,3,i,2,I,...,2,i,3,I,2,I,4,NaN,2.571429,0.786796
29,30,Q1400881,property,Q2574811,cause,sibling,4,NaN,4,NaN,...,4,NaN,4,NaN,2,S,4,NaN,3.714286,0.755929


# Fetch parents for siblings dataset and create triplets for sentence construction

In [7]:
p279SibWordSet = set(p279_SiblingsDF.node1.to_list() + p279_SiblingsDF.node2.to_list())
p279SibWordPairSet = set(p279_SiblingsDF.node1.to_list() + p279_SiblingsDF.node2.to_list())

In [8]:
p279_Parents_DF = pd.read_csv('../data/P279_dataset/wikidata-P279.tsv',sep='\t')

In [9]:
p279_Parents_DF.head()

,id,node1,label,node2,rank,node2;wikidatatype
0,Q100000030-P279-Q14748-30394205-0,Q100000030,P279,Q14748,normal,wikibase-item
1,Q100000058-P279-Q1622444-bd182663-0,Q100000058,P279,Q1622444,normal,wikibase-item
2,Q1000032-P279-Q1813494-0aa0f1dc-0,Q1000032,P279,Q1813494,normal,wikibase-item
3,Q1000032-P279-Q83602-482a1943-0,Q1000032,P279,Q83602,normal,wikibase-item
4,Q1000039-P279-Q11555767-2dddfd86-0,Q1000039,P279,Q11555767,normal,wikibase-item


In [11]:
p279_Parents_DF1 = p279_Parents_DF[p279_Parents_DF.apply(lambda p: p.node1 in p279SibWordSet, axis=1)]

In [12]:
labelsDF = pd.read_csv('../data/labels.en.tsv', sep='\t')

In [15]:
temp1 = p279_Parents_DF1.set_index('node1').join(labelsDF.set_index('node1'), rsuffix='_label')

In [16]:
temp1 = temp1[['id','label','node2','node2_label']]

In [19]:
temp2 = temp1.reset_index().set_index('node2').join(labelsDF.set_index('node1'), rsuffix='_label2')

In [20]:
temp3 = temp2[['id','node1','label','node2', 'node2_label']].rename(columns={'node2_label': 'node1_label', 'node2':'node2_label'}).reset_index().rename(columns={'index':'node2'})

In [21]:
temp3 = temp3[['id','node1','node1_label','label','node2','node2_label']]

In [22]:
temp3 = temp3[~temp3['node1_label'].isna()]
temp3 = temp3[~temp3['node2_label'].isna()]

In [23]:
temp3['node2_label'].isna().sum()

0

In [24]:
temp3['node1_label'] = temp3['node1_label'].apply(lambda p: p[1:-4])

In [25]:
temp3['node2_label'] = temp3['node2_label'].apply(lambda p: p[1:-4])

In [26]:
temp3

,id,node1,node1_label,label,node2,node2_label
0,Q523-P279-Q1027098-3ce5cd2f-0,Q523,star,P279,Q1027098,fusor
1,Q381072-P279-Q104493-9eab93d0-0,Q381072,crisis,P279,Q104493,risk
2,Q176763-P279-Q11028-28192af9-0,Q176763,evidence,P279,Q11028,information
3,Q3695082-P279-Q11028-fb23389c-0,Q3695082,sign,P279,Q11028,information
4,Q551800-P279-Q11105360-c1431a2e-0,Q551800,ticket,P279,Q11105360,medium of exchange
...,...,...,...,...,...,...
96,Q951964-P279-Q8195619-98e34f68-0,Q951964,food product,P279,Q8195619,human food
97,Q30461-P279-Q82955-2d27e026-0,Q30461,president,P279,Q82955,politician
98,Q951964-P279-Q868404-10500678-0,Q951964,food product,P279,Q868404,final good
99,Q2006324-P279-Q930933-06286d46-0,Q2006324,agreement,P279,Q930933,relation


In [27]:
pairLeftDF = temp3.copy()
pairRightDF = temp3.copy()

In [28]:
pairsDF = pairLeftDF.set_index('node2').join(pairRightDF.set_index('node2'),rsuffix='_right').reset_index()

In [29]:
pairsDF['set_nodes'] = pairsDF.apply(lambda p: str(sorted([p.node1, p.node1_right]) + [p.node2]), axis=1)

In [30]:
pairsDF1 = pairsDF[~pairsDF['set_nodes'].duplicated()]

In [31]:
print(f"{len(pairsDF)} statements get reduced to {len(pairsDF1)} statements after checking for duplicate pairs")

196 statements get reduced to 148 statements after checking for duplicate pairs


In [32]:
pairsDF1.head()

,node2,id,node1,node1_label,label,node2_label,id_right,node1_right,node1_label_right,label_right,node2_label_right,set_nodes
0,Q1027098,Q523-P279-Q1027098-3ce5cd2f-0,Q523,star,P279,fusor,Q523-P279-Q1027098-3ce5cd2f-0,Q523,star,P279,fusor,"['Q523', 'Q523', 'Q1027098']"
1,Q104493,Q381072-P279-Q104493-9eab93d0-0,Q381072,crisis,P279,risk,Q381072-P279-Q104493-9eab93d0-0,Q381072,crisis,P279,risk,"['Q381072', 'Q381072', 'Q104493']"
2,Q11028,Q176763-P279-Q11028-28192af9-0,Q176763,evidence,P279,information,Q176763-P279-Q11028-28192af9-0,Q176763,evidence,P279,information,"['Q176763', 'Q176763', 'Q11028']"
3,Q11028,Q176763-P279-Q11028-28192af9-0,Q176763,evidence,P279,information,Q3695082-P279-Q11028-fb23389c-0,Q3695082,sign,P279,information,"['Q176763', 'Q3695082', 'Q11028']"
5,Q11028,Q3695082-P279-Q11028-fb23389c-0,Q3695082,sign,P279,information,Q3695082-P279-Q11028-fb23389c-0,Q3695082,sign,P279,information,"['Q3695082', 'Q3695082', 'Q11028']"


In [ ]:
pairsDF1[pairsDF1.apply(lambda p: (p.node1, p.node2) in )]